how this notebook works

In [1]:
# dependencies

import pandas as pd
import sqlalchemy as sq
import sys, os
import pickle
from imblearn.combine import SMOTEENN, SMOTETomek
from xgboost import XGBRFClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
)
from imblearn.ensemble import (
    BalancedRandomForestClassifier,
)

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
)

sys.path.append("../../")
os.chdir("../../")
from ModelBuilderMethods import getConn, extractYears, scaleColumns, encodeColumns
from Models.models_ensemble import getBalancedClassifier, getClassifier

In [2]:
# unlimited line output
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 500)

setting up a dataset

In [3]:
weatherStationQuery = sq.text(
    """
    SELECT * from dataset_cross_monthly_station
"""
)

weatherSatQuery = sq.text(
    """
    SELECT * from dataset_cross_monthly_sat
"""
)

ergotPrevYearsAggQuery = sq.text(
    """
    SELECT year, district, 
    present_prev1, present_prev2, present_prev3,
    percnt_true_prev1, percnt_true_prev2, percnt_true_prev3 
    from agg_ergot_sample_v2
"""
)

ergotTargetQuery = sq.text(
    """
    SELECT year, district, downgrade from ergot_sample_feat_eng
"""
)

In [4]:
conn = getConn("./.env")

# stationDf = pd.read_sql(weatherStationQuery, conn)
satelliteDf = pd.read_sql(weatherSatQuery, conn)
ergotPrevDf = pd.read_sql(ergotPrevYearsAggQuery, conn)
ergotTargetDf = pd.read_sql(ergotTargetQuery, conn)

conn.close()
del conn

In [5]:
# merge on year and district
tempdf = pd.merge(satelliteDf, ergotPrevDf, on=["year", "district"], how="left")
del satelliteDf
del ergotPrevDf
# tempdf = satelliteDf
# tempdf = stationDf

# merge on year and district
datasetDf = pd.merge(ergotTargetDf, tempdf, on=["year", "district"], how="left")
del ergotTargetDf
del tempdf

categorical values [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)  


In [6]:

# encode district
datasetDf["district"] = datasetDf["district"].astype("category")

temp = pd.get_dummies(datasetDf["district"], prefix="district", drop_first=True)
datasetDf = pd.concat([datasetDf, temp], axis=1)

datasetDf = datasetDf.drop(columns=["district"])

del temp

splitting the dataset

In [7]:
# train 1995 - 2015 test 2016 - 2020
trainDf = extractYears(datasetDf, 1995, 2015)
testDf = extractYears(datasetDf, 2016, 2020)
del datasetDf

In [8]:
# drop year
trainDf = trainDf.drop(columns=["year"])
testDf = testDf.drop(columns=["year"])


balancing the dataset https://imbalanced-learn.org/stable/



In [9]:
# pre balancing check
# print value counts downgrade
print(trainDf["downgrade"].value_counts())
print(testDf["downgrade"].value_counts())

downgrade
False    122202
True       2082
Name: count, dtype: int64
downgrade
False    26307
True      1016
Name: count, dtype: int64


In [10]:
# count nan
print(trainDf.isna().sum())
# set nan to 0
# trainDf = trainDf.fillna(0)

# drop nan
trainDf = trainDf.dropna()

downgrade                           0
1:min_dewpoint_temperature          0
1:min_temperature                   0
1:min_evaporation_from_bare_soil    0
1:min_skin_reservoir_content        0
                                   ..
district_4830                       0
district_4840                       0
district_4850                       0
district_4860                       0
district_4870                       0
Length: 687, dtype: int64


In [11]:
balancer = SMOTEENN(sampling_strategy=1, random_state=42)
balancedTrainDfX, balancedTrainDfY = balancer.fit_resample(
    trainDf.drop(columns="downgrade"), trainDf["downgrade"]
)

In [12]:
# post balancing check
# print value counts downgrade
print(balancedTrainDfY.value_counts())

downgrade
False    115239
True      25156
Name: count, dtype: int64


### normalization / scaling
some blurb about scalers  
0 [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)             
1 [MaxAbsScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html)  
2 [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)  
3 [RobustScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)  
4 [Normalizer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html)  
5 [PowerTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html)  
6 [QuantileTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.QuantileTransformer.html)  

In [13]:
# df = pd.DataFrame()
# scaled = scaleColumns(df, ['max_temp'], None, 1)

categorical values [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)  


In [14]:
# encoded = encodeColumns(df, ['max_temp'], None)

In [15]:
def printMetrics(model_name, y_true, y_pred):
    print(model_name)
    print("Accuracy: ", accuracy_score(y_true, y_pred))
    print("Precision: ", precision_score(y_true, y_pred))
    print("Recall: ", recall_score(y_true, y_pred))
    print("F1: ", f1_score(y_true, y_pred))
    print("ROC AUC: ", roc_auc_score(y_true, y_pred))
    print("Classification Report: \n", classification_report(y_true, y_pred))
    print()

selecting a model

In [16]:
ESTIMATORS = 400
DEPTH = 40
CORES = 10
MINSPLSPLIT = 8
MINSAMPLELEAF = 4

model_rf = RandomForestClassifier(
    n_estimators=ESTIMATORS,
    random_state=42,
    max_depth=DEPTH,
    n_jobs=CORES,
    min_samples_split=MINSPLSPLIT,
    min_samples_leaf=MINSAMPLELEAF,
)
model_nobalance_rf = RandomForestClassifier(
    n_estimators=ESTIMATORS,
    random_state=42,
    max_depth=DEPTH,
    n_jobs=CORES,
    min_samples_split=MINSPLSPLIT,
    min_samples_leaf=MINSAMPLELEAF,
)
balanced_model_rf = BalancedRandomForestClassifier(
    n_estimators=ESTIMATORS,
    random_state=42,
    max_depth=DEPTH,
    n_jobs=CORES,
    min_samples_split=MINSPLSPLIT,
    min_samples_leaf=MINSAMPLELEAF,
)
balanced_model_balanced_rf = BalancedRandomForestClassifier(
    n_estimators=ESTIMATORS,
    random_state=42,
    max_depth=DEPTH,
    n_jobs=CORES,
    min_samples_split=MINSPLSPLIT,
    min_samples_leaf=MINSAMPLELEAF,
)
model_xgbrf = XGBRFClassifier(
    n_estimators=ESTIMATORS,
    random_state=42,
    max_depth=DEPTH,
    n_jobs=CORES,
)
model_balance_xgbrf = XGBRFClassifier(
    n_estimators=ESTIMATORS,
    random_state=42,
    max_depth=DEPTH,
    n_jobs=CORES,
)

In [17]:
model_nobalance_rf.fit(trainDf.drop(columns="downgrade"), trainDf["downgrade"])
model_rf.fit(balancedTrainDfX, balancedTrainDfY)
balanced_model_rf.fit(trainDf.drop(columns="downgrade"), trainDf["downgrade"])
balanced_model_balanced_rf.fit(balancedTrainDfX, balancedTrainDfY)
model_xgbrf.fit(trainDf.drop(columns="downgrade"), trainDf["downgrade"])
model_balance_xgbrf.fit(balancedTrainDfX, balancedTrainDfY)

/usr/local/lib/python3.8/dist-packages/imblearn/ensemble/_forest.py:546: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/usr/local/lib/python3.8/dist-packages/imblearn/ensemble/_forest.py:558: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/usr/local/lib/python3.8/dist-packages/imblearn/ensemble/_forest.py:546: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/usr/local/lib/p

XGBRFClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bytree=None,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
                grow_policy=None, importance_type=None,
                interaction_constraints=None, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=40, max_leaves=None,
                min_child_weight=None, missing=nan, monotone_constraints=None,
                n_estimators=400, n_jobs=10, num_parallel_tree=None,
                objective='binary:logistic', predictor=None, random_state=42,
                reg_alpha=None, ...)

eval procedure

In [18]:
# set nan to 0
# testDf = testDf.fillna(0)

# drop nan
testDf = testDf.dropna()

In [19]:
# get predictions
predictions = model_rf.predict(testDf.drop(columns="downgrade"))
predictions_nobalance = model_nobalance_rf.predict(testDf.drop(columns="downgrade"))
predictions_balanced = balanced_model_rf.predict(testDf.drop(columns="downgrade"))
predictions_balanced_balanced = balanced_model_balanced_rf.predict(
    testDf.drop(columns="downgrade")
)
predictions_xgbrf = model_xgbrf.predict(testDf.drop(columns="downgrade"))
predictions_balance_xgbrf = model_balance_xgbrf.predict(
    testDf.drop(columns="downgrade")
)

In [20]:
print(pd.DataFrame(predictions).value_counts())
print(pd.DataFrame(predictions_nobalance).value_counts())
print(pd.DataFrame(predictions_balanced).value_counts())
print(pd.DataFrame(predictions_balanced_balanced).value_counts())
print(pd.DataFrame(predictions_xgbrf).value_counts())
print(pd.DataFrame(predictions_balance_xgbrf).value_counts())

False    22907
True      4416
Name: count, dtype: int64
False    27323
Name: count, dtype: int64
False    26237
True      1086
Name: count, dtype: int64
False    25565
True      1758
Name: count, dtype: int64
0    27323
Name: count, dtype: int64
0    22679
1     4644
Name: count, dtype: int64


print model performance metrics on test data

In [21]:
printMetrics("sk RF balanced train set", testDf["downgrade"], predictions)
printMetrics("sk RF imbalanced train set", testDf["downgrade"], predictions_nobalance)
printMetrics("imb RF imbalanced train set", testDf["downgrade"], predictions_balanced)
printMetrics(
    "imb RF balanced train set", testDf["downgrade"], predictions_balanced_balanced
)
printMetrics("xgb RF imbalanced train set", testDf["downgrade"], predictions_xgbrf)
printMetrics(
    "xgb RF balanced train set", testDf["downgrade"], predictions_balance_xgbrf
)

sk RF balanced train set
Accuracy:  0.8126120850565458
Precision:  0.035326086956521736
Recall:  0.15354330708661418
F1:  0.05743740795287187
ROC AUC:  0.49580461055094766
Classification Report: 
               precision    recall  f1-score   support

       False       0.96      0.84      0.90     26307
        True       0.04      0.15      0.06      1016

    accuracy                           0.81     27323
   macro avg       0.50      0.50      0.48     27323
weighted avg       0.93      0.81      0.86     27323


sk RF imbalanced train set
Accuracy:  0.9628152106284082
Precision:  0.0
Recall:  0.0
F1:  0.0
ROC AUC:  0.5
Classification Report: 
               precision    recall  f1-score   support

       False       0.96      1.00      0.98     26307
        True       0.00      0.00      0.00      1016

    accuracy                           0.96     27323
   macro avg       0.48      0.50      0.49     27323
weighted avg       0.93      0.96      0.94     27323


imb RF imbala

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: Undefine

Recall:  0.1377952755905512
F1:  0.04946996466431096
ROC AUC:  0.48329304586156974
Classification Report: 
               precision    recall  f1-score   support

       False       0.96      0.83      0.89     26307
        True       0.03      0.14      0.05      1016

    accuracy                           0.80     27323
   macro avg       0.50      0.48      0.47     27323
weighted avg       0.93      0.80      0.86     27323


